# Introduçãos às CNNs no PyTorch

## Objetivos

Este notebook contém exemplo numérico de uma rede com uma camada convolucional e uma camada densa. A camada convolucional possui ativação reLU e max-pooling. Já a camada densa possui uma única saída com ativação sigmóide.

Com esse exemplo, aprende-se a:
- construir a rede utilizando subclasses de Module do PyTorch
- inicializar os pesos e biases da rede convolucional e da rede densa
- visualizar os dados intermediários da rede

## Importação dos Módulos

In [2]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.functional as F
from torch.autograd import Variable

from collections import OrderedDict
from IPython.core.display import display, HTML

np.set_printoptions(precision=3) # ponto flutuante com 3 casas para facilitar a impressão

In [3]:
# verifica se a GPU está disponível
use_gpu = torch.cuda.is_available()
print("Usando GPU:", use_gpu)

Usando GPU: True


## Definição da Rede, camadas convolucionais e densas

### Definição de camadas

Em redes neurais, uma camada é usualmente um neurônio, que inclui a soma de multiplicação de
pesos ou convolução e uma ativação usualmente não linear. Podemos dizer que o max-pooling 
também faz parte da camada.

A rede que iremos utilizar neste exemplo possui 2 camadas: uma convolucional e outra densa.
A camada convolucional terá ativação reLU e um max-pooling, já a camada densa terá uma
ativação sigmóide.

### Rede a ser implementada

![alt text](https://raw.githubusercontent.com/vcasadei/images/master/RedeIntroKeras.png)

In [21]:
print(3*1*2*2 + 2*2 + 3*2*2)

28


### Criando redes np PyTorch

Uma CNN no PyTorch é criada utilizando uma classe que é subclasse do *torch.nn.Module*. Essa classe poderá ter variáveis que também são instâncias de subclasses de *torch.nn.Module*.

- *torch.nn.Conv2d*
- *torch.nn.ReLU*
- *torch.nn.MaxPool2d*
- *torch.nn.Linear*
- *torch.nn.Sequential*

Todas estas classes são também subclasses de *torch.nn.Module* e podem ser instanciadas no contrutor para serem utilizadas na classe (ver [Documentação torch.nn](http://pytorch.org/docs/master/nn.html))

### Implementação do modelo

In [4]:
class Model(nn.Module):
    
    # Definição de Modules que serão utilizados na rede
    def __init__(self):
        super(Model, self).__init__()
        
        # entrada a)  shape (1,6,6)
        # b) entrada com shape (3,6,6)
        
        # 4 filtros de saída e kernel (3,3)
        
        
        # camada convolucional
        self.conv = nn.Conv2d(3, 3, (3, 2), padding=0, bias=True)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(3, 4, (3, 3), padding=0, bias=True)
        # 4, 2, 3
        self.max_pool = nn.MaxPool2d((2, 2))
        
        
        # camada densa
        self.dense = nn.Linear(4, 1, bias=True)
        self.sigmoid = nn.Sigmoid()
    
    # Método de definição obrigatória
    # Sequencia que será efetivamente executada para obter a saída da rede
    def forward(self, x):
        # passa os dados pela camada convolucional
        x = self.conv(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.max_pool(x)
        
        
        # faz o flatten dos dados
        x = x.view(-1, 4)
        
        # passa os dados pela camada densa
        x = self.dense(x)
        x = self.sigmoid(x)
        
        return x
    
    # Método opcional utilizado para mostrar a saída de cada parte da rede
    def modules_output(self, x):
        self.output = OrderedDict()        

        # passa os dados pela camada convolucional
        x = self.conv(x)
        self.output['conv'] = x
        
        x = self.relu(x)
        self.output['relu'] = x
        
        x = self.conv2(x)
        self.output['conv2'] = x
        
        x = self.max_pool(x)
        self.output['max_pool'] = x
        
        
        
        # faz o flatten dos dados
        print(x.shape)
        x = x.view(-1, 4)
        self.output['flatten'] = x
        
        # passa os dados pela camada densa
        x = self.dense(x)
        self.output['dense'] = x
        x = self.sigmoid(x)
        self.output['sigmoid'] = x

        return self.output

In [5]:
model = Model()
print(model)

Model(
  (conv): Conv2d(3, 3, kernel_size=(3, 2), stride=(1, 1))
  (relu): ReLU()
  (conv2): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1))
  (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (dense): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


### Outro modelo encapsulando as camadas em uma *Sequential*

In [6]:
class Model2(nn.Module):

    # Definição de Modules que serão utilizados na rede
    def __init__(self):
        super(Model2, self).__init__()
        
        # camada convolucional
        self.cnn = nn.Sequential(OrderedDict([
            ('conv', nn.Conv2d(1, 3, (2, 2), padding=0, bias=True)),
            ('relu', nn.ReLU()),
            ('max_pool', nn.MaxPool2d((2, 2)))
        ]))
        
        # camada densa
        self.dense = nn.Sequential(OrderedDict([
            ('dense', nn.Linear(3*4, 1, bias=True)),
            ('sigmoid', nn.Sigmoid())
        ]))

    # Método de definição obrigatória
    # Sequencia que será efetivamente executada para obter a saída da rede
    def forward(self, x):
        # passa os dados pela camada convolucional
        x = self.cnn(x)
        # faz o flatten dos dados
        x = x.view(-1, 4)
        # passa os dados pela camada densa
        x = self.dense(x) 
        
        return x

In [7]:
model2 = Model2()
print(model2)

Model2(
  (cnn): Sequential(
    (conv): Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
    (relu): ReLU()
    (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (dense): Linear(in_features=12, out_features=1, bias=True)
    (sigmoid): Sigmoid()
  )
)


## Criação dos parâmetros da rede

Neste exemplo não haverá treinamento, iremos inicializar os parâmetros da rede com valores conhecidos, de acordo com a figura.

### Kernel da convolução

In [8]:
# número de filtros
n_filters = 3    

# comprimento e largura dos filtros
k_height = 3
k_width = 2 
kernel_shape = (k_height, k_width)

W_conv = np.array([[[1,2],
                    [3,4],
                    [1,1]],
                  [[5,6],
                   [7,8],
                   [1,1]],
                  [[9,10],
                   [11,12],
                   [1,1]],
                  [[1,2],
                    [3,4],
                    [1,1]],
                  [[5,6],
                   [7,8],
                   [1,1]],
                  [[9,10],
                   [11,12],
                   [1,1]],
                  [[1,2],
                    [3,4],
                    [1,1]],
                  [[5,6],
                   [7,8],
                   [1,1]],
                  [[9,10],
                   [11,12],
                   [1,1]]]).reshape(n_filters,3,k_height,k_width)

In [9]:
# número de filtros   

# comprimento e largura dos filtros
k_height = 3
k_width = 3
kernel_shape = (k_height, k_width)

W_conv2 = np.array([[[1,2, 1],
                    [3,4,1],
                    [1,1,1]],
                  [[5,6,1],
                   [7,8,1],
                   [1,1,1]],
                  [[9,10,1],
                   [11,12,1],
                   [1,1,1]],
                  [[1,2,1],
                    [3,4,1],
                    [1,1,1]],
                  [[5,6,1],
                   [7,8,1],
                   [1,1,1]],
                  [[9,10,1],
                   [11,12,1],
                   [1,1,1]],
                  [[1,2,1],
                    [3,4,1],
                    [1,1,1]],
                  [[5,6,1],
                   [7,8,1],
                   [1,1,1]],
                  [[9,10,1],
                   [11,12,1],
                   [1,1,1]],
                   [[1,2,1],
                    [3,4,1],
                    [1,1,1]],
                  [[5,6,1],
                   [7,8,1],
                   [1,1,1]],
                  [[9,10,1],
                   [11,12,1],
                   [1,1,1]]]).reshape(4,3,k_height,k_width)

### Bias da Convolução

In [10]:
# valor de bias
f_bias = 0.1     
bias_conv = np.arange(1,3+1) * f_bias
print("Bias da convolução:",bias_conv)

Bias da convolução: [0.1 0.2 0.3]


In [11]:
# valor de bias
f_bias = 0.1     
bias_conv2 = np.arange(1,4+1) * f_bias
print("Bias da convolução:",bias_conv2)

Bias da convolução: [0.1 0.2 0.3 0.4]


### Pesos para a camada densa

In [12]:
# após o max pooling, são 3 imagens 2x2 = 12
W_dense = np.arange(4).reshape(1, 4)
print("Pesos da camada densa:\n",W_dense)

Pesos da camada densa:
 [[0 1 2 3]]


### Bias para a camada densa

In [13]:
bias_dense = np.ones(1) * f_bias
print("Bias da camanda densa:",bias_dense)

Bias da camanda densa: [0.1]


### Sumário do shape dos parâmetros

In [14]:
# Mostra o shape dos pesos das camadas
display(HTML('<h3>Shape dos pesos</h3>'))
print('{:11} {}'.format('W_conv:', W_conv.shape))
print('{:11} {}'.format('bias_conv:', bias_conv.shape))
print('{:11} {}'.format('W_dense:', W_dense.shape))
print('{:11} {}'.format('bias_dense:', bias_dense.shape))
print()

W_conv:     (3, 3, 3, 2)
bias_conv:  (3,)
W_dense:    (1, 4)
bias_dense: (1,)



## Criação da rede model e carregamento dos pesos (parâmetros)

In [15]:
# cria o modelo
model = Model()

# Dicionário com os pesos para a rede
my_weights = OrderedDict([
    ('conv.weight',  torch.FloatTensor(W_conv.astype(float))),
    ('conv.bias',    torch.FloatTensor(bias_conv.astype(float))),
    ('conv2.weight',  torch.FloatTensor(W_conv2.astype(float))),
    ('conv2.bias',    torch.FloatTensor(bias_conv2.astype(float))),
    ('dense.weight', torch.FloatTensor( W_dense.astype(float))),
    ('dense.bias',   torch.FloatTensor(bias_dense.astype(float))),
])

# aplica os pesos criados à rede
model.load_state_dict(my_weights)

<All keys matched successfully>

### Visualização dos pesos da rede na forma de dicionário

In [16]:
model.state_dict()

OrderedDict([('conv.weight',
              tensor([[[[ 1.,  2.],
                        [ 3.,  4.],
                        [ 1.,  1.]],
              
                       [[ 5.,  6.],
                        [ 7.,  8.],
                        [ 1.,  1.]],
              
                       [[ 9., 10.],
                        [11., 12.],
                        [ 1.,  1.]]],
              
              
                      [[[ 1.,  2.],
                        [ 3.,  4.],
                        [ 1.,  1.]],
              
                       [[ 5.,  6.],
                        [ 7.,  8.],
                        [ 1.,  1.]],
              
                       [[ 9., 10.],
                        [11., 12.],
                        [ 1.,  1.]]],
              
              
                      [[[ 1.,  2.],
                        [ 3.,  4.],
                        [ 1.,  1.]],
              
                       [[ 5.,  6.],
                        [ 7.,  8.],

### Visualização dos pesos da rede na forma de tensores

In [17]:
for W_name, W in model.state_dict().items():
    display(HTML('<h4>{}</h4>'.format(W_name)))
    print(W)

tensor([[[[ 1.,  2.],
          [ 3.,  4.],
          [ 1.,  1.]],

         [[ 5.,  6.],
          [ 7.,  8.],
          [ 1.,  1.]],

         [[ 9., 10.],
          [11., 12.],
          [ 1.,  1.]]],


        [[[ 1.,  2.],
          [ 3.,  4.],
          [ 1.,  1.]],

         [[ 5.,  6.],
          [ 7.,  8.],
          [ 1.,  1.]],

         [[ 9., 10.],
          [11., 12.],
          [ 1.,  1.]]],


        [[[ 1.,  2.],
          [ 3.,  4.],
          [ 1.,  1.]],

         [[ 5.,  6.],
          [ 7.,  8.],
          [ 1.,  1.]],

         [[ 9., 10.],
          [11., 12.],
          [ 1.,  1.]]]])


tensor([0.1000, 0.2000, 0.3000])


tensor([[[[ 1.,  2.,  1.],
          [ 3.,  4.,  1.],
          [ 1.,  1.,  1.]],

         [[ 5.,  6.,  1.],
          [ 7.,  8.,  1.],
          [ 1.,  1.,  1.]],

         [[ 9., 10.,  1.],
          [11., 12.,  1.],
          [ 1.,  1.,  1.]]],


        [[[ 1.,  2.,  1.],
          [ 3.,  4.,  1.],
          [ 1.,  1.,  1.]],

         [[ 5.,  6.,  1.],
          [ 7.,  8.,  1.],
          [ 1.,  1.,  1.]],

         [[ 9., 10.,  1.],
          [11., 12.,  1.],
          [ 1.,  1.,  1.]]],


        [[[ 1.,  2.,  1.],
          [ 3.,  4.,  1.],
          [ 1.,  1.,  1.]],

         [[ 5.,  6.,  1.],
          [ 7.,  8.,  1.],
          [ 1.,  1.,  1.]],

         [[ 9., 10.,  1.],
          [11., 12.,  1.],
          [ 1.,  1.,  1.]]],


        [[[ 1.,  2.,  1.],
          [ 3.,  4.,  1.],
          [ 1.,  1.,  1.]],

         [[ 5.,  6.,  1.],
          [ 7.,  8.,  1.],
          [ 1.,  1.,  1.]],

         [[ 9., 10.,  1.],
          [11., 12.,  1.],
          [ 1.,  1.,  1.]]]

tensor([0.1000, 0.2000, 0.3000, 0.4000])


tensor([[0., 1., 2., 3.]])


tensor([0.1000])


## Predição da Rede e Visualização dos tensores em cada camada

### Imagem de Entrada: Variable (1,1,5,6) uma amostra, um canal, 5 linhas e 6 colunas

In [18]:
X = np.array([[[[0,0,0,0, 0,0],
                [0,0,1,0, 0,0],
                [0,0,0,0, 0,0],
                [0,0,0,0,-1,0],
                [0,0,0,0, 0,0],
                [0,0,0,0, 0,0]],
              [[0,0,0,0, 0,0],
                [0,0,1,0, 0,0],
                [0,0,0,0, 0,0],
                [0,0,0,0,-1,0],
                [0,0,0,0, 0,0],
                [0,0,0,0, 0,0]],
              [[0,0,0,0, 0,0],
                [0,0,1,0, 0,0],
                [0,0,0,0, 0,0],
                [0,0,0,0,-1,0],
                [0,0,0,0, 0,0],
                [0,0,0,0, 0,0]]]])

display(HTML('<h3>Entrada na rede</h3>'))
display(HTML('<h4>X</h4>'))
X_tensor = torch.from_numpy(X).type(torch.FloatTensor)
X_var = Variable (X_tensor)
print(X_var)

tensor([[[[ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.]]]])


## Predição

In [19]:
y_pred = model.modules_output(X_var)

torch.Size([1, 4, 1, 1])


### Mostra as saídas de cada camada da rede

In [20]:
display(HTML('<h3>Saídas de cada camada da rede</h3>'))

# Visualização do tensor na saída de cada camada
for module_name, output in y_pred.items():
    display(HTML('<h4>{}</h4>'.format(module_name)))
    print(output)

tensor([[[[  0.1000,  24.1000,  21.1000,   0.1000,   0.1000],
          [  0.1000,  18.1000,  15.1000,  -2.9000,  -2.9000],
          [  0.1000,   0.1000,   0.1000, -23.9000, -20.9000],
          [  0.1000,   0.1000,   0.1000, -17.9000, -14.9000]],

         [[  0.2000,  24.2000,  21.2000,   0.2000,   0.2000],
          [  0.2000,  18.2000,  15.2000,  -2.8000,  -2.8000],
          [  0.2000,   0.2000,   0.2000, -23.8000, -20.8000],
          [  0.2000,   0.2000,   0.2000, -17.8000, -14.8000]],

         [[  0.3000,  24.3000,  21.3000,   0.3000,   0.3000],
          [  0.3000,  18.3000,  15.3000,  -2.7000,  -2.7000],
          [  0.3000,   0.3000,   0.3000, -23.7000, -20.7000],
          [  0.3000,   0.3000,   0.3000, -17.7000, -14.7000]]]],
       grad_fn=<ConvolutionBackward0>)


tensor([[[[ 0.1000, 24.1000, 21.1000,  0.1000,  0.1000],
          [ 0.1000, 18.1000, 15.1000,  0.0000,  0.0000],
          [ 0.1000,  0.1000,  0.1000,  0.0000,  0.0000],
          [ 0.1000,  0.1000,  0.1000,  0.0000,  0.0000]],

         [[ 0.2000, 24.2000, 21.2000,  0.2000,  0.2000],
          [ 0.2000, 18.2000, 15.2000,  0.0000,  0.0000],
          [ 0.2000,  0.2000,  0.2000,  0.0000,  0.0000],
          [ 0.2000,  0.2000,  0.2000,  0.0000,  0.0000]],

         [[ 0.3000, 24.3000, 21.3000,  0.3000,  0.3000],
          [ 0.3000, 18.3000, 15.3000,  0.0000,  0.0000],
          [ 0.3000,  0.3000,  0.3000,  0.0000,  0.0000],
          [ 0.3000,  0.3000,  0.3000,  0.0000,  0.0000]]]],
       grad_fn=<ReluBackward0>)


tensor([[[[ 993.8998, 1496.6998,  644.4999],
          [ 390.9000,  560.1000,  234.5000]],

         [[ 993.9999, 1496.7998,  644.5999],
          [ 391.0000,  560.2000,  234.6000]],

         [[ 994.0999, 1496.8999,  644.6999],
          [ 391.1000,  560.3000,  234.7000]],

         [[ 994.1999, 1496.9999,  644.7999],
          [ 391.2000,  560.4000,  234.8000]]]], grad_fn=<ConvolutionBackward0>)


tensor([[[[1496.6998]],

         [[1496.7998]],

         [[1496.8999]],

         [[1496.9999]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)


tensor([[1496.6998, 1496.7998, 1496.8999, 1496.9999]], grad_fn=<ViewBackward0>)


tensor([[8981.6992]], grad_fn=<AddmmBackward0>)


tensor([[1.]], grad_fn=<SigmoidBackward0>)


# Atividades

1. Quais as vantagens de utilizar Sequential na definição do modelo?
2. Como calcular o número de parâmetros da rede?
3. Retirar o bias e recalcular o número de parâmetros a serem treinados
4. A rede foi projetada para aceitar entrada com shape (1,5,6). Mudar a rede para aceitar:
   a) entrada com shape (1,6,6)
   b) entrada com shape (3,6,6)
5. Inserir mais uma camada convolucional com 4 filtros de saída e kernel (3,3)

# Referências

- [PyTorch Documentation](http://pytorch.org/docs/master/index.html)


# Aprendizados com este notebook
